In [1]:
params_d = {
    "number_of_iterations": [
        10,
        20,
        30,
        40,
        50,
        60,
        70,
        80,
        90,
        100,
        110,
        120,
        130,
        140,
        150,
        160,
        170,
        180,
        190,
        200,
    ],
    "number_of_pivots": [
        5,
        10,
        15,
        20,
        25,
        30,
        35,
        40,
        45,
        50,
        55,
        60,
        65,
        70,
        75,
        80,
        85,
        90,
        95,
        100,
    ],
    "eps": [
        0.05,
        0.1,
        0.15,
        0.2,
        0.25,
        0.3,
        0.35,
        0.4,
        0.45,
        0.5,
        0.55,
        0.6,
        0.65,
        0.7,
        0.75,
        0.8,
        0.85,
        0.9,
        0.95,
        1.0,
    ],
}

In [2]:
import pandas as pd
from config.paths import root_path
from config.quality_metrics import qm_names
from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from config.paths import root_path

In [3]:
dataset_names = ["les_miserables", "1138_bus", "USpowerGrid"]
params_names = sorted(["number_of_pivots", "number_of_iterations", "eps"])
qm_names_ss = [f"{qm_name}_ss" for qm_name in qm_names]

In [4]:
data = pd.DataFrame()

for dataset_name in dataset_names:
    data_path = root_path.joinpath(
        f"data/experiments/regression_analysis/grid/{dataset_name}.pkl"
    )
    datum = pd.read_pickle(data_path)
    data = pd.concat([data, datum])

In [5]:
data['aspect_ratio'] =  -data['aspect_ratio']
data['neighborhood_preservation'] =  -data['neighborhood_preservation']
data['crossing_number'] =  -data['crossing_number']
data['crossing_angle'] =  -data['crossing_angle']
data['crossing_angle'] =  -data['crossing_angle']
data['gabriel_graph_property'] =  -data['gabriel_graph_property']
data['ideal_edge_lengths'] =  -data['ideal_edge_lengths']
data['node_resolution'] =  -data['node_resolution']
data['stress'] =  -data['stress']

In [6]:
data

,angular_resolution,aspect_ratio,crossing_angle,crossing_number,gabriel_graph_property,ideal_edge_lengths,neighborhood_preservation,node_resolution,stress,number_of_pivots,number_of_iterations,eps,dataset_name
data_id,,,,,,,,,,,,,
bb2b7119-883e-4089-b4be-ec98b2150af5,995.080139,0.643333,1003.676880,-2363.0,-9.997450e+04,-44.471745,0.270000,-2.262040e+04,-503.152740,5,10,0.05,les_miserables
4e9ce796-cd3d-48d0-a551-59dfc4c7b4b3,1029.114502,0.781833,995.103638,-2506.0,-8.457808e+04,-34.284576,0.209524,-1.489409e+04,-543.295227,5,10,0.05,les_miserables
5f4303bd-4857-47b3-bc66-664737b0acd7,977.956604,0.762826,827.940247,-1980.0,-6.019839e+04,-40.869736,0.372973,-2.084025e+04,-340.642731,5,10,0.05,les_miserables
3c22943f-fe59-4f84-ac0f-d6d70f0c9259,1008.232727,0.820159,915.304993,-2173.0,-7.827785e+04,-49.901482,0.349270,-2.451729e+04,-393.484222,5,10,0.05,les_miserables
891e3ff8-8a70-4e41-a1b9-b2aed4773038,1011.226013,0.962789,968.700562,-2514.0,-8.352752e+04,-44.770088,0.286076,-2.164487e+04,-443.041748,5,10,0.05,les_miserables
...,...,...,...,...,...,...,...,...,...,...,...,...,...
a83b2b88-140d-499b-8183-569259bf8f71,6797.813477,0.886762,17081.269531,-32263.0,-1.407859e+07,-4406.994141,0.045754,-9.360683e+09,-725598.687500,100,200,0.80,USpowerGrid
b74f4d5e-21b9-4a1d-a8c6-661abfc9f0e8,6873.985352,0.889589,16705.615234,-30398.0,-1.053275e+07,-3451.234619,0.043643,-8.086209e+09,-736127.187500,100,200,0.85,USpowerGrid
f946aa86-82b5-4260-8ec3-a4a6fc931e17,6669.931641,0.870054,17320.759766,-36757.0,-1.607190e+07,-5071.155762,0.040720,-8.936648e+09,-727798.062500,100,200,0.90,USpowerGrid


In [7]:

kf = KFold(n_splits=5, shuffle=True, random_state=0)
sss = {
    dataset_name: {qm_name: StandardScaler() for qm_name in qm_names}
    for dataset_name in dataset_names
}

size = [2400, 24000, 240000]

mean_squared_errors_train = []
mean_squared_errors_test = []
zansa_train = {}
zansa_test = {}
for qm_name in qm_names:
    pass
zansa_data = []


data[qm_names_ss] = 0

sdata = data

for train_index, test_index in kf.split(data):
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]
    for dataset_name in dataset_names:
        dd = train_data.query(f'dataset_name == "{dataset_name}"')
        for qm_name in qm_names:
            qd = dd[qm_name]
            sss[dataset_name][qm_name].fit(qd.values.reshape(-1, 1))
    for dataset_name in dataset_names:
        dd = data.query(f'dataset_name == "{dataset_name}"')
        for qm_name in qm_names:
            qd = dd[qm_name]
            data.loc[dd.index, [f"{qm_name}_ss"]] = sss[dataset_name][
                qm_name
            ].transform(qd.values.reshape(-1, 1))

    reg = MultiOutputRegressor(estimator=lgb.LGBMRegressor())

    # 更新されないから更新
    train_data = data.iloc[train_index]
    test_data = data.iloc[test_index]
    x_train, x_test = train_data[params_names], test_data[params_names]
    y_train, y_test = train_data[qm_names_ss], test_data[qm_names_ss]

    reg.fit(x_train, y_train)

    y_train_pred = reg.predict(x_train)
    y_test_pred = reg.predict(x_test)

    zansa_r = {"type": "train"}
    for qm_name_ss in qm_names_ss:
        zansa_r[qm_name_ss] = 0
    for (i, row), train_p in zip(y_train.iterrows(), y_train_pred):
        d = row.to_dict()
        for qm_name_ss, p in zip(qm_names_ss, train_p):
            zansa_r[qm_name_ss] += (d[qm_name_ss] - p) ** 2
    for qm_name_ss in qm_names_ss:
        zansa_r[qm_name_ss] = zansa_r[qm_name_ss] / len(y_train_pred)
    zansa_data.append(zansa_r)

    zansa_r = {"type": "test"}
    for qm_name_ss in qm_names_ss:
        zansa_r[qm_name_ss] = 0
    for (i, row), test_p in zip(y_test.iterrows(), y_test_pred):
        d = row.to_dict()
        for qm_name_ss, p in zip(qm_names_ss, test_p):
            zansa_r[qm_name_ss] += (d[qm_name_ss] - p) ** 2
    for qm_name_ss in qm_names_ss:
        zansa_r[qm_name_ss] = zansa_r[qm_name_ss] / len(y_test_pred)
    zansa_data.append(zansa_r)

In [8]:
zansa_df = pd.DataFrame(zansa_data)
display(zansa_df.sort_values(["type"]))

,type,angular_resolution_ss,aspect_ratio_ss,crossing_angle_ss,crossing_number_ss,gabriel_graph_property_ss,ideal_edge_lengths_ss,neighborhood_preservation_ss,node_resolution_ss,stress_ss
1,test,0.647494,0.805143,0.346500,0.397216,0.654636,0.705639,0.348409,0.936687,0.327841
3,test,0.636132,0.789012,0.324362,0.372840,0.680011,0.717249,0.339165,0.917703,0.316250
5,test,0.648703,0.786153,0.326467,0.376217,0.651514,0.740737,0.344032,0.932380,0.339651
7,test,0.640883,0.798774,0.337744,0.390252,0.667677,0.732580,0.342145,0.924295,0.319419
9,test,0.640362,0.796588,0.318788,0.369112,0.830234,0.769998,0.345092,0.922558,0.348345
0,train,0.635395,0.784878,0.324002,0.374160,0.665965,0.709864,0.340424,0.916042,0.319383
2,train,0.635832,0.782863,0.326487,0.375997,0.662869,0.706896,0.341387,0.914820,0.318108
4,train,0.633287,0.779799,0.325536,0.374919,0.667150,0.704142,0.339971,0.916572,0.315793
6,train,0.634442,0.782374,0.323646,0.372988,0.664824,0.711270,0.340288,0.916721,0.317473
8,train,0.632372,0.781653,0.325461,0.374936,0.658120,0.700879,0.338628,0.915462,0.315005


In [9]:
params_list = []
p = params_d
for noi in p["number_of_iterations"]:
    for nop in p["number_of_pivots"]:
        for eps in p["eps"]:
            params_list.append([eps, noi, nop])

pred_results = reg.predict(params_list)
reg_result_df_data = []
for params, pred_result in zip(params_list, pred_results):
    r = {
        "eps": params[0],
        "number_of_iterations": params[1],
        "number_of_pivots": params[2],
    }
    for qm_name, pred_r in zip(qm_names, pred_result):
        r[qm_name] = pred_r
    reg_result_df_data.append(r)

reg_result_df = pd.DataFrame(reg_result_df_data)

In [10]:
reg_result_df

,eps,number_of_iterations,number_of_pivots,angular_resolution,aspect_ratio,crossing_angle,crossing_number,gabriel_graph_property,ideal_edge_lengths,neighborhood_preservation,node_resolution,stress
0,0.05,10,5,-2.302718,-0.728702,5.022824,-5.334505,-11.536813,-11.462237,-2.846549,2.149697,-5.122097
1,0.10,10,5,-2.052873,-0.323713,4.548969,-4.891856,-12.089994,-10.494406,-2.838186,1.910954,-4.880969
2,0.15,10,5,-1.802145,-0.875364,3.269002,-3.713139,-9.076440,-7.582573,-2.760647,1.828834,-3.943662
3,0.20,10,5,-1.760885,-0.708166,3.328291,-3.714335,-5.648601,-5.973490,-2.690482,1.845299,-3.651433
4,0.25,10,5,-1.641806,-1.112834,3.615871,-4.001355,-5.994264,-6.027373,-2.705959,1.896061,-4.454587
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,0.80,200,100,-0.484905,0.324678,-0.573116,0.549855,0.175859,-0.130770,0.878938,-0.108307,0.440450
7996,0.85,200,100,-0.481513,0.268241,-0.515438,0.460825,0.113371,-0.173859,0.861029,-0.057716,0.424905
7997,0.90,200,100,-0.469647,0.418042,-0.431430,0.369829,0.064981,-0.283549,0.832557,-0.123484,0.410704
7998,0.95,200,100,-0.462170,0.343111,-0.399967,0.269855,-0.019497,-0.380882,0.852140,-0.028357,0.393325


In [11]:
reg_result_df.query("eps==0.5 & number_of_pivots==100")

,eps,number_of_iterations,number_of_pivots,angular_resolution,aspect_ratio,crossing_angle,crossing_number,gabriel_graph_property,ideal_edge_lengths,neighborhood_preservation,node_resolution,stress
389,0.5,10,100,-1.768301,0.130897,-0.389066,-0.021770,0.019763,-0.247233,0.457884,0.075102,0.417785
789,0.5,20,100,-0.963961,0.267961,-0.667973,0.557518,0.261210,0.029245,0.687765,-0.094679,0.473788
1189,0.5,30,100,-0.718768,0.303069,-0.736148,0.693727,0.325336,0.121562,0.760374,-0.147559,0.492471
1589,0.5,40,100,-0.609764,0.315142,-0.774705,0.747654,0.345463,0.165308,0.778206,-0.154633,0.501851
1989,0.5,50,100,-0.575280,0.296012,-0.784479,0.764571,0.367915,0.215015,0.804675,-0.194244,0.504558
2389,0.5,60,100,-0.572947,0.305419,-0.813717,0.797023,0.380266,0.225002,0.836579,-0.195650,0.509628
2789,0.5,70,100,-0.551586,0.314391,-0.818584,0.805943,0.389623,0.244043,0.842153,-0.194628,0.515074
3189,0.5,80,100,-0.515203,0.319991,-0.842810,0.824475,0.392253,0.259210,0.844745,-0.194628,0.517115
3589,0.5,90,100,-0.518842,0.320283,-0.865157,0.835341,0.403622,0.262713,0.837648,-0.206800,0.517115
3989,0.5,100,100,-0.506103,0.320283,-0.865157,0.835878,0.404356,0.256467,0.873682,-0.208141,0.516489


In [13]:
NAME_ABBREVIATIONS = {
    "angular_resolution": "ANR",
    "aspect_ratio": "AR",
    "crossing_angle": "CA",
    "crossing_number": "CN",
    "gabriel_graph_property": "GB",
    "ideal_edge_lengths": "IE",
    "node_resolution": "NR",
    "run_time": "RT",
    "neighborhood_preservation": "SB",
    "stress": "ST",
}

In [14]:
p_abb = {
    "number_of_pivots": "ピボットの数",
    "number_of_iterations": "反復回数",
    "eps": "ε",
}

In [15]:
from itertools import combinations
import japanize_matplotlib

cs = list(combinations(params_names, 2))
ALPHA = 0.5

for c in cs:
    px = c[0]
    py = c[1]

    pe = list(filter(lambda x: x != px and x != py, params_names))[0]
    pxvs = params_d[px]
    pyvs = params_d[py]
    pevs = params_d[pe]

    for qm_name in qm_names:
        for pev in pevs:
            fig = plt.figure(figsize=(8, 8))
            axis = fig.add_subplot(111, projection="3d")
            fig.subplots_adjust(left=0.04, right=0.90, bottom=0.05, top=0.95)
            axis.set_title(
                f"{NAME_ABBREVIATIONS[qm_name]} {p_abb[pe]}={pev}",
                fontsize=28,
            )
            axis.set_xlabel(
                p_abb[px],
                fontsize=20,
            )
            axis.set_ylabel(
                p_abb[py],
                fontsize=20,
            )
            axis.set_zlabel(NAME_ABBREVIATIONS[qm_name], fontsize=20)
            axis.invert_xaxis()
            # px固定してpy回す
            for pxv in pxvs:
                # for pev in pevs:
                y = np.array(pyvs).reshape(-1, 1)
                x = np.array([pxv] * len(pyvs)).reshape(-1, 1)
                # z = reg.predict(y)
                z = np.array(
                    reg_result_df.query(f"{px}=={pxv} & {pe}=={pev}")[qm_name]
                ).reshape(-1, 1)

                axis.plot(x, y, z, alpha=ALPHA, color="blue")

            # py固定してpx回す
            for pyv in pyvs:
                # for pev in pevs:
                x = np.array(pxvs).reshape(-1, 1)
                y = np.array([pyv] * len(pxvs)).reshape(-1, 1)
                # z = regr.predict(x)
                z = np.array(
                    reg_result_df.query(f"{py}=={pyv} & {pe}=={pev}")[qm_name]
                ).reshape(-1, 1)

                axis.plot(x, y, z, alpha=ALPHA, color="blue")

            export_path = (
                root_path.joinpath("202309mid")
                .joinpath("3d_fixed")
                .joinpath(f"{dataset_name}-{qm_name}-{px}-{py}-{pe}={pev}.png")
            )
            export_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(export_path)
            plt.close()

In [16]:
# params importanceの可視化
data = [
    {
        "評価指標": NAME_ABBREVIATIONS["angular_resolution"],
        "number_of_pivots": 212958.454,
        "number_of_iterations": 134951.673,
        "eps": 23587.686,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["aspect_ratio"],
        "number_of_pivots": 209613.788,
        "number_of_iterations": 6177.569,
        "eps": 4854.362,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["crossing_angle"],
        "number_of_pivots": 626540.032,
        "number_of_iterations": 12736.544,
        "eps": 42362.591,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["crossing_number"],
        "number_of_pivots": 526700.816,
        "number_of_iterations": 47110.003,
        "eps": 57832.451,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["gabriel_graph_property"],
        "number_of_pivots": 204384.155,
        "number_of_iterations": 104520.279,
        "eps": 36573.899,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["ideal_edge_lengths"],
        "number_of_pivots": 102624.975,
        "number_of_iterations": 91471.732,
        "eps": 108172.929,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["neighborhood_preservation"],
        "number_of_pivots": 604783.748,
        "number_of_iterations": 5046.919,
        "eps": 58502.994,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["node_resolution"],
        "number_of_pivots": 67056.782,
        "number_of_iterations": 11118.033,
        "eps": 7252.902,
    },
    {
        "評価指標": NAME_ABBREVIATIONS["stress"],
        "number_of_pivots": 677945.218,
        "number_of_iterations": 7495.615,
        "eps": 6764.718,
    },
]

for d in data:
    s = d["number_of_pivots"] + d["number_of_iterations"] + d["eps"]
    d["反復回数"] = d["number_of_iterations"] / s
    d["ε"] = d["eps"] / s
    # d["ピボットの数"] = 1 - (d["反復回数"] + d["ε"])
    d["ピボットの数"] = d["number_of_pivots"] / s

imp_df = pd.DataFrame(data)

In [17]:
import plotly.express as px

tick_labels = ["ピボットの数", "反復回数", "ε"]

fig = px.bar(
    imp_df,
    x="評価指標",
    y=["ピボットの数", "反復回数", "ε"],
    # color="viridis",
    title="HPの重要度（割合）",
)

fig.update_layout(yaxis=dict(title="重要度"))


fig.write_image(root_path.joinpath('202309mid/imp_fixed/imp.png'), width=1000, height=400, scale=3)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/fuga_takata/dev/vdslab-project/hyperparameter_optimization/202309mid/imp_fixed/imp.png'